<a href="https://colab.research.google.com/github/BStricks/SeaShips/blob/master/image_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir("drive/My Drive/Colab Notebooks/evello")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import torch
import torchvision
from torchvision import transforms
from torchvision.transforms import ToTensor
from PIL import Image
import xml.etree.ElementTree as ET
import io 
from shapely.geometry import box


device = torch.device('cpu')

def prediction(image, XML):    
  #JPEG -> TENSOR
  tensor = image_to_tensor(image)
  if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
  else:
    map_location=device
  model = torch.load("./data/SeaShips/full_model.pth", map_location=map_location)
  model = model.to(device)           
  #get model prediction and XML data in JSON
  output = JSON_output(tensor, XML, model)    
  return output
 
def JSON_output(tensor,XML,model):
  cats = {0:'bulk cargo carrier',1:'container ship',2:'fishing boat',3:'general cargo ship',4:'ore carrier',5:'passenger ship'}
  tree = ET.parse(XML)
  root = tree.getroot()
  TIMESTAMP = root.find('./TIMESTAMP').text
  LATITUDE = root.find('./LAT').text
  LONGITUDE = root.find('./LON').text 
  model.eval()
 
  with torch.no_grad():
    prediction = model([tensor.to(device)])
    try:  
      json_dict = {'ships':[],'timestamp':TIMESTAMP,'latitude':LATITUDE,'longitude':LONGITUDE}
      for i in range(0, len(prediction[0]['labels'])):
        labels = prediction[0]['labels'][i].item()
        label = cats.get(labels)
        scores = prediction[0]['scores'][i].item()
        score = "{:.2f}%".format(scores*100)
        x_min =  "{:.0f}".format(prediction[0]['boxes'][i][0].item())
        y_min =  "{:.0f}".format(prediction[0]['boxes'][i][1].item())
        x_max =  "{:.0f}".format(prediction[0]['boxes'][i][2].item())
        y_max =  "{:.0f}".format(prediction[0]['boxes'][i][3].item())
        ship = {'score':score,'x_min':x_min,'y_min':y_min,'x_max':x_max,'y_max':y_max,'label':label}
        json_dict['ships'].append(ship)
    
    except:
      json_dict = {'score':None,'x_min':None,'y_min':None,'x_max':None,'y_max':None,'label':None,'timestamp':TIMESTAMP,'location':LATITUDE,'longitude':LONGITUDE}
  
  return json_dict
 
def image_to_tensor(image_data):
    image = Image.open(image_data)
    transform = transforms.Compose([ToTensor()])
    tensor = transform(image)
    return tensor

In [0]:
def get_file_list():
  path = "data/SeaShips/XML"
  path2 = "data/SeaShips/images"
  XML_list = []
  image_list = []
  for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    filename2 = filename.split('-')[1]
    filename2 = filename2.split('.')[0]
    filename2 = filename2+'.jpg'
    filename2 = os.path.join(path2,filename2)
    image_list.append(filename2)
    fullname = os.path.join(path, filename)
    XML_list.append(fullname)
  return XML_list, image_list

In [0]:
XML_list, image_list = get_file_list()

In [5]:
device = torch.device('cpu')
results = []
for i in range(0,len(XML_list)):
  #image = Image.open(image_list[i])
  result = prediction(image_list[i],XML_list[i])
  results.append(result)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)


In [0]:
import json

ais_list = []
ais_list_matched = []
path = "./data/SeaShips/AIS"

for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r') as json_file:
      data = json.load(json_file)
      ais_list.append(data)

def match_ais(xml):
  tree = ET.parse(xml)
  root = tree.getroot()
  TIMESTAMP = root.find('./TIMESTAMP').text
  for i in ais_list:
    if i['TIMESTAMP_UTC'] == TIMESTAMP:
      return i

for i in XML_list:
  ais = match_ais(i)
  ais_list_matched.append(ais)



In [8]:
!pip install pomegranate
from pomegranate import *

     |████████████████████████████████| 3.3MB 4.2MB/s 
  Created wheel for pomegranate: filename=pomegranate-0.12.2-cp36-cp36m-linux_x86_64.whl size=10958123 sha256=3d6b4a12b5e2da5d1fe4239419f2c53fadb830227b3cb49de6c227081c4c7543
  Stored in directory: /root/.cache/pip/wheels/51/14/41/7285a60a08951abde03ce60f47a8644dd96d8e22b373bbbcc8
Successfully built pomegranate


In [0]:
from shapely.geometry import box

def generateEvent(ais, image=None, text=None):
  #read in jsons and get characteristics
  if ais['PORT_NAME'] == 'SAN DIEGO':

    if -120.3442 <= ais['LAT'] <= -117.6526 and 32.3707 <= ais['LON'] <= 33.2933 :
      position = 'A'
    else:
      position = 'N'
    
    if 0 <= ais['SPEED'] <= 0.8 :
      speed = 'A'
    else:
      speed = 'N'

    if 290 <= ais['HEADING'] <= 350 :
      heading = 'A'
    else:
      heading = 'N'

  elif ais['PORT_NAME'] == 'NORFOLK':

    if -75.7837 <= ais['LAT'] <= -75.0256 and 35.7457 <= ais['LON'] <= 36.7917 :
      position = 'A'
    else:
      position = 'N'
    
    if 0 <= ais['SPEED'] <= 0.8 :
      speed = 'A'
    else:
      speed = 'N'

    if 300 <= ais['HEADING'] <= 340 :
      heading = 'A'
    else:
      heading = 'N'
  
  if image:
    if len(image['ships']) > 1:
      polygon_list = []
      boat_overlap = 'N'

      for bbox in image['ships']:
        x_min = int(bbox['x_min'])
        y_min = int(bbox['y_min'])
        x_max = int(bbox['x_max'])
        y_max = int(bbox['y_max'])
        rect = box(x_min, y_min, x_max, y_max, ccw=True)
        polygon_list.append(rect)

        for i in range(0, len(polygon_list)):
          for a in range(i+1,len(polygon_list)):
              distance = polygon_list[i].distance(polygon_list[a])
              if distance <= 350.0:
                boat_overlap = 'A'
    else:
      boat_overlap = 'N'

  return heading, speed, position, boat_overlap

In [0]:
def bayes_network(heading, speed, position, boat_overlap):
  heading_dis = DiscreteDistribution({'A': 1./5, 'N': 4./5})
  speed_dis = DiscreteDistribution({'A': 1./5, 'N': 4./5})
  position_dis = DiscreteDistribution({'A': 2./5, 'N': 3./5})
  boat_overlap_dis = DiscreteDistribution({'A': 4./5, 'N': 1./5})

  event_dis = ConditionalProbabilityTable(
      [['A', 'A', 'A', 'A', 'A', 1],
       ['A', 'A', 'A', 'A', 'N', 0],
       ['A', 'A', 'A', 'N', 'A', 0.9],
       ['A', 'A', 'A', 'N', 'N', 0.1],
       ['A', 'A', 'N', 'A', 'A', 0.8],
       ['A', 'A', 'N', 'A', 'N', 0.2],
       ['A', 'N', 'A', 'A', 'A', 0.8],
       ['A', 'N', 'A', 'A', 'N', 0.2],
       ['N', 'A', 'A', 'A', 'A', 0.85],
       ['N', 'A', 'A', 'A', 'N', 0.15],
       ['N', 'N', 'A', 'A', 'A', 0.7],
       ['N', 'N', 'A', 'A', 'N', 0.3],
       ['N', 'A', 'N', 'A', 'A', 0.7],
       ['N', 'A', 'N', 'A', 'N', 0.3],
       ['N', 'A', 'A', 'N', 'A', 0.6],
       ['N', 'A', 'A', 'N', 'N', 0.4],
       ['A', 'N', 'N', 'A', 'A', 0.75],
       ['A', 'N', 'N', 'A', 'N', 0.25],
       ['A', 'N', 'A', 'N', 'A', 0.7],
       ['A', 'N', 'A', 'N', 'N', 0.3],
       ['A', 'A', 'N', 'N', 'A', 0.7],
       ['A', 'A', 'N', 'N', 'N', 0.3],
       ['A', 'N', 'N', 'N', 'A', 0.4],
       ['A', 'N', 'N', 'N', 'N', 0.6],
       ['N', 'A', 'N', 'N', 'A', 0.45],
       ['N', 'A', 'N', 'N', 'N', 0.65],
       ['N', 'N', 'A', 'N', 'A', 0.3],
       ['N', 'N', 'A', 'N', 'N', 0.7],
       ['N', 'N', 'N', 'A', 'A', 0.45],
       ['N', 'N', 'N', 'A', 'N', 0.55],
       ['N', 'N', 'N', 'N', 'A', 0],
       ['N', 'N', 'N', 'N', 'N', 1]], [heading_dis, speed_dis, position_dis, boat_overlap_dis])
  
  s1 = Node(heading_dis, name="heading")
  s2 = Node(speed_dis, name="speed")
  s3 = Node(position_dis, name="position")
  s4 = Node(boat_overlap_dis, name="boat_overlap")
  s5 = Node(event_dis, name="event")

  model = BayesianNetwork("ship_event_reasoning")
  model.add_states(s1, s2, s3, s4, s5)
  model.add_edge(s1, s5)
  model.add_edge(s2, s5)
  model.add_edge(s3, s5)
  model.add_edge(s4, s5)
  model.bake()
  pred = model.predict([[heading, speed, position, boat_overlap, None]])
  if pred[0][4] == 'N':
    non_pred = 'A'
  elif pred[0][4] == 'A':
    non_pred = 'N'
  prob1 = model.probability(pred)
  prob2 = model.probability([[heading, speed, position, boat_overlap, non_pred]])
  prob = prob1/(prob1+prob2)
  prob = "{:.2f}%".format(prob*100)
  return pred, prob, heading, speed, position, boat_overlap


In [0]:
def event_reasoning(ais, image=None, text=None):  
  position, speed, heading, boat_overlap = generateEvent(ais,image)
  print(position, speed, heading, boat_overlap)
  pred, prob, heading, speed, position, boat_overlap = bayes_network(position, speed, heading, boat_overlap)
  return {'prediction':pred[0][4], 'probability':prob}, heading, speed, position, boat_overlap

In [23]:
for i in range(0,300):
  dict1, heading, speed, position, boat_overlap = event_reasoning(ais=ais_list_matched[i],image=results[i])
  print(dict1)

N N A N
{'prediction': 'N', 'probability': '70.00%'}
N N N N
{'prediction': 'N', 'probability': '100.00%'}
N A A N
{'prediction': 'A', 'probability': '60.00%'}
N N N N
{'prediction': 'N', 'probability': '100.00%'}
N N N N
{'prediction': 'N', 'probability': '100.00%'}
N N N N
{'prediction': 'N', 'probability': '100.00%'}
N N N N
{'prediction': 'N', 'probability': '100.00%'}
N N N N
{'prediction': 'N', 'probability': '100.00%'}
N N A N
{'prediction': 'N', 'probability': '70.00%'}
N N N N
{'prediction': 'N', 'probability': '100.00%'}
A N A N
{'prediction': 'A', 'probability': '70.00%'}
N N A N
{'prediction': 'N', 'probability': '70.00%'}
N N N N
{'prediction': 'N', 'probability': '100.00%'}
N A A N
{'prediction': 'A', 'probability': '60.00%'}
N N A N
{'prediction': 'N', 'probability': '70.00%'}
N N N N
{'prediction': 'N', 'probability': '100.00%'}
N N N N
{'prediction': 'N', 'probability': '100.00%'}
N N A N
{'prediction': 'N', 'probability': '70.00%'}
N N A N
{'prediction': 'N', 'probabi